In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
np.random.seed(1232)
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\dep')
data = pd.read_csv("Rand_train.csv", sep=',')
data_val = pd.read_csv("Rand_valid.csv", sep=',')

In [3]:
data.cost = (np.exp(data.cost) - 0.5)*((data.cost != 0) * 1)
data_val.cost = (np.exp(data_val.cost) - 0.5)*((data_val.cost != 0) * 1)
data = shuffle(data).reset_index(drop=True)
data = data.drop(['fmde', 'lxghindx', 'lpi'], axis=1)
data_val = data_val.drop(['fmde', 'lxghindx', 'lpi'], axis=1)

In [4]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

ranking = np.load('ranking.npy')

my_index = X.columns.values[ranking]
X2 = X[my_index]
X2_val = X_val[my_index]

In [5]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [6]:
os.chdir('C:\\Users\\rober\\Desktop\\RAND_pro\\prog_calc')
from fit5_nn import fit

In [15]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = F.relu(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        #x3 = torch.stack((x1[:,0], x2[:,0]), dim=1) 
        x3 = torch.cat((x1, x2), 1)
        return x3

In [16]:
def comb_error(output, target):
    output = (output + 0.0001)*0.999
    logErr1 = -1 * torch.log(output) * target[:,0].view(1,-1).t()
    logErr2 = -1 * torch.log(1 - output) * (1 - target[:,0].view(1,-1)).t()
    mseErr = torch.pow(target - output, 2) * (1 - target[:,0].view(1,-1)).t()
    return (1/output.shape[0]) * (torch.sum(logErr1,dim=0)[0] + torch.sum(logErr2,dim=0)[0] + (1/(2*950000))*torch.sum(mseErr,dim=0)[1])

In [17]:
n_epochs = 5000
lr = 0.001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 10
clipping = 0.20
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining7.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
net = Net1()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

#PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining7.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=comb_error, 
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

Epoch 1: Training Loss: 1.2290415664513905, Validation Loss: 1.3301912546157837
Epoch 2: Training Loss: 1.2261294921239216, Validation Loss: 1.325783133506775
Epoch 3: Training Loss: 1.2210701902707417, Validation Loss: 1.319699764251709
Epoch 4: Training Loss: 1.2145775258541107, Validation Loss: 1.3124592304229736
Epoch 5: Training Loss: 1.206950028737386, Validation Loss: 1.3044452667236328
Epoch 6: Training Loss: 1.198581059773763, Validation Loss: 1.2959372997283936
Epoch 7: Training Loss: 1.1898809671401978, Validation Loss: 1.2871400117874146
Epoch 8: Training Loss: 1.1809752782185872, Validation Loss: 1.2782007455825806
Epoch 9: Training Loss: 1.1719586253166199, Validation Loss: 1.2692264318466187
Epoch 10: Training Loss: 1.1628147264321644, Validation Loss: 1.2602943181991577
Epoch 11: Training Loss: 1.1538567741711934, Validation Loss: 1.251458764076233
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 1.1449500024318695, Validation Loss: 1.242760419845581
Epoch 13: Tr

KeyboardInterrupt: 

In [ ]:
PATH = 'C:\\Users\\rober\\Desktop\\RAND_pro\\Data\\checkpoints\\mytraining7.pt'
initial = X2.shape[1]
final = int(round(initial * 1.5, 0)) 
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [ ]:
with torch.no_grad():
    val_inputs = torch.FloatTensor(X2_val.values)
    val_labels = torch.FloatTensor(Y2_val)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
    val_outputs = net.forward(val_inputs)
    val_loss = comb_error(val_outputs, val_labels) 
    print(val_loss.item())

In [ ]:
val_outputs

In [ ]:
val_labels

In [ ]:
val_outputs = val_outputs.numpy()
val_labels = val_labels.numpy()
y_scores = val_outputs[:,0]
y_real = val_labels[:,0]

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    plt.figure(figsize=(8,8))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation=90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.legend(loc='best')

In [ ]:
fpr, tpr, auc_thresholds = roc_curve(y_real, y_scores)
print(auc(fpr, tpr)) # AUC of ROC
plot_roc_curve(fpr, tpr, 'recall_optimized')

In [ ]:
y = val_labels[:,1][val_labels[:,1]>0]
yhat = val_outputs[:,1][val_labels[:,1]>0]

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
y = np.abs(val_labels[:,1])
p = val_outputs[:,0]
f2 = val_outputs[:,1]
yhat = (1-p)*f2

In [ ]:
RMSE = np.sqrt(np.mean((y - yhat)**2))
RMSE

In [ ]:
print(np.cov(y, p))

In [ ]:
print(np.cov(f2, p))